In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

in_channel = 3
num_classes = 10
learning_rate = 1e-3
batch_size = 1024
num_epochs = 5

model = torchvision.models.googlenet(pretrained= True)
for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Linear(1024, num_classes)
model.to(device)
train_dataset = datasets.CIFAR10(root='dataset/',
                                 train=True,
                                 transform=transforms.ToTensor(),
                                 download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Files already downloaded and verified


In [5]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor= 0.1, patience=5, verbose=True)


In [6]:
for epoch in range(num_epochs):
    losses = []
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)

        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    mean_loss = sum(losses)/len(losses)
    scheduler.step(mean_loss)
    print(f"epoch : {epoch}    cost : {mean_loss}")

epoch : 0    cost : 2.092412043590935
epoch : 1    cost : 1.8707834603835125
epoch : 2    cost : 1.807168415614537
epoch : 3    cost : 1.7753893331605561
epoch : 4    cost : 1.7580375574073013
